To do:
-Try operations for a single year's data first (all preprocessing steps)

Preprocessing notes (single year, multi years, etc)

Filtration based on shootings procedure:

(1*) Retain any municipality where a shooting occurred ever (when working with a single year, it doesn't really matter, just filter based on shooting)

Over many years, this may include places with changed crime profiles, which is bad




(2*) Retain all data for a year in a municipality when there was a shooting that year (same as option 1* for a single year's data)

Over many years, this means retaining all samples from the year where the shooting ocurred for that municipality; this has the benefit of interpretability: every sample is just a troubled year municipality

Post column creation:

Need to make post-1, post-2, ... , post-6 month indicator columns (initialize to 0)

Figure out how to do this efficiently LOL

Pre column creation:

Need to make a pre (treatment) column (post = -1 in estimating equation) (use shift=-1, think about whether min/max needs to be changed)


Column transformations

Total assaults, total killings
    
z-scored force sizes, force size as a percentage of population column (check if it exists first)

Join arrests, calculate total arrests, take log of arrests as new response variable

Relevant summary statistics/notes to add in the introduction?

Number of municipalities, number of municipalities preserved per year (on average) after filtration, number of officers shot and killed (on average) per year

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore

In [84]:
leoka_17_monthly = pd.read_stata('ucr_leoka_monthly_1960_2020_dta/leoka_monthly_2017.dta')
leoka_17_monthly.shape
leoka_17_monthly = leoka_17_monthly.iloc[:,:-198] #drop unnecessary columns

In [85]:
leoka_17_monthly['total_assaults'] = leoka_17_monthly['assaults_no_injury_total'] + leoka_17_monthly['assaults_with_injury_total']

#filter into only the municipalities where there was at least one assault that year
officer_assaulted = (leoka_17_monthly['total_assaults'] > 0)
ori_to_keep = leoka_17_monthly.loc[officer_assaulted, 'ori'].unique()
leoka_17_monthly = leoka_17_monthly[leoka_17_monthly['ori'].isin(ori_to_keep)]

#leoka_17_monthly.shape --> 66372 x 59, means 5531 municipalities were kept


In [86]:
for i in range(7):
    leoka_17_monthly[f'PreviousTreatment_{i}'] = leoka_17_monthly.groupby('ori')['officers_killed_total'].shift(i)
    leoka_17_monthly[f'post_{i}'] = leoka_17_monthly[f'PreviousTreatment_{i}'] > 0

    #don't do the idxmin thing, shouldn't matter since things are NaN where edge case shifts; drop the NaNs by removing the PreviousTreament_i columns

    leoka_17_monthly[f'post_{i}'] = leoka_17_monthly[f'post_{i}'].astype(int)
    leoka_17_monthly = leoka_17_monthly.drop(columns=[f'PreviousTreatment_{i}'])

In [89]:
#column transformations

#force size mechanism construction
min_size, max_size = leoka_17_monthly['total_employees_officers'].min(), leoka_17_monthly['total_employees_officers'].max()
leoka_17_monthly['employment_norm'] = (leoka_17_monthly['total_employees_officers'] - min_size)/(max_size - min_size) #defaults to 0-1 range

median_condition = leoka_17_monthly['total_employees_officers'] > leoka_17_monthly['total_employees_officers'].median()
trueval, falseval = 1, 0
leoka_17_monthly['employment_median_indicator'] = np.where(median_condition, trueval, falseval)

leoka_17_monthly['employment_pop_proportion'] = leoka_17_monthly['total_employees_officers']/leoka_17_monthly['population']
prop_median_condition = leoka_17_monthly['employment_pop_proportion'] > leoka_17_monthly['employment_pop_proportion'].median()
leoka_17_monthly['employment_pop_prop_indicator'] = np.where(prop_median_condition, trueval, falseval)